In [1]:
import numpy as np
import pandas as pd

## Birbirine bağlama
Her iki kaynak aynı formatta ise **pd.concat()** yöntemini kullanmaktır

*concatenation** yöntemi basitce sütunlarla veya satırlarla iki DataFrame'i yapıştırır

**Aynı formatta olan iki DataFrame'i birbirine yapıştırma**